In [114]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns; sns.set()  # for plot styling
import locale
from difflib import SequenceMatcher

In [2]:
archivo_marca1= pd.read_excel("Informacion Bases de datos Comercial Marca 1.xlsx", sheet_name=None)
archivo_marca2_10= pd.read_excel("Informacion Bases de datos Comercial Marca 2 - 10.xlsx", sheet_name=None)

In [3]:
archivo_marca2_10.keys()

dict_keys(['Marca 2', 'Marca 3', 'Marca 4', 'Marca 5', 'Marca 6', 'Marca 7', 'marca 8', 'marca 9', 'marca 10'])

In [4]:
def sacar_anio(entrada):
    return entrada.year
def sacar_mes(entrada):
    return entrada.month
def sacar_dia(entrada):
    return entrada.day

In [5]:
def obt_horas_minutos_marca4(entrada):
    entrada=  entrada.replace(".0","")
    return entrada.split(":")[1] + ":"+ entrada.split(":")[2]

In [6]:
def obt_horas_marca8(entrada):
    return entrada.split(":")[0]
def obt_horas_minutos_marca8(entrada):
    return entrada[:-3]

In [7]:
def obt_horas_minutos_marca10(entrada):
    return entrada.split(":")[1] + ":"+ entrada.split(":")[2]

In [8]:
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')

'es_ES.UTF-8'

In [9]:
marca1=archivo_marca1['MARCA 1'].copy()
marca1=marca1.rename(columns={'Fecha de Venta':'fecha_de_venta','Centro de consumo':'tienda','Ventas Netas':'ventas','ZONA':'muelle','HORA':'hora_entera','TRANSACCIONES':'transacciones','SEMANA':'semana','Año':'anio_entero','Mes':'mes_entero','Día':'dia_entero','Mes(2)':'mes_string'})
marca1['marca'] = 'marca1' 
marca1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83370 entries, 0 to 83369
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   fecha_de_venta  83370 non-null  datetime64[ns]
 1   tienda          83370 non-null  object        
 2   ventas          83370 non-null  int64         
 3   muelle          83370 non-null  object        
 4   hora_entera     83282 non-null  float64       
 5   transacciones   83282 non-null  float64       
 6   semana          83370 non-null  int64         
 7   anio_entero     83370 non-null  int64         
 8   mes_entero      83370 non-null  int64         
 9   dia_entero      83370 non-null  int64         
 10  mes_string      83370 non-null  object        
 11  marca           83370 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(5), object(4)
memory usage: 7.6+ MB


In [10]:
marca2=archivo_marca2_10['Marca 2'].copy()
marca2=marca2.drop(columns=['Unnamed: 7','Unnamed: 8','Unnamed: 9'])
marca2=marca2.rename(columns={'Fecha':'fecha_de_venta','Tienda':'tienda','Ventas':'ventas','Muelle':'muelle','Hora entera':'hora_entera','Transacciones':'transacciones','Unidades':'unidades'})
marca2['marca'] = 'marca2' 
marca2['anio_entero'] = marca2['fecha_de_venta'].apply(sacar_anio)
marca2['mes_entero'] = marca2['fecha_de_venta'].apply(sacar_mes)
marca2['dia_entero'] = marca2['fecha_de_venta'].apply(sacar_dia)
marca2['mes_string'] = marca2['fecha_de_venta'].dt.strftime('%B')
marca2['semana'] = marca2['fecha_de_venta'].dt.week
# dates.dt.week
marca2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22297 entries, 0 to 22296
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tienda          22297 non-null  object        
 1   hora_entera     22297 non-null  int64         
 2   fecha_de_venta  22297 non-null  datetime64[ns]
 3   ventas          22297 non-null  float64       
 4   transacciones   22297 non-null  int64         
 5   unidades        22297 non-null  int64         
 6   muelle          22297 non-null  object        
 7   marca           22297 non-null  object        
 8   anio_entero     22297 non-null  int64         
 9   mes_entero      22297 non-null  int64         
 10  dia_entero      22297 non-null  int64         
 11  mes_string      22297 non-null  object        
 12  semana          22297 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(7), object(4)
memory usage: 2.2+ MB


In [11]:
marca2.head()

,tienda,hora_entera,fecha_de_venta,ventas,transacciones,unidades,muelle,marca,anio_entero,mes_entero,dia_entero,mes_string,semana
0,Tienda A,7,2022-01-01,27121380.0,7,23,Internacional Oriente,marca2,2022,1,1,enero,52
1,Tienda A,8,2022-01-01,4636320.0,3,5,Internacional Oriente,marca2,2022,1,1,enero,52
2,Tienda A,9,2022-01-01,10518300.0,4,12,Internacional Oriente,marca2,2022,1,1,enero,52
3,Tienda A,10,2022-01-01,9620520.0,6,16,Internacional Oriente,marca2,2022,1,1,enero,52
4,Tienda A,11,2022-01-01,22212840.0,7,21,Internacional Oriente,marca2,2022,1,1,enero,52


In [12]:
marca4=archivo_marca2_10['Marca 4'].copy()
marca4=marca4.rename(columns={'FECHA':'fecha_de_venta','TIENDA1':'tienda','TOTAL_LOC':'ventas','Muelle':'muelle','HORA ENTERA':'hora_entera','HORA':'hora_minutos','TRANSACCIONES':'transacciones','UNIDADES':'unidades','CATEGORÍA':'categoria','Num fact':'num_fact','DESCRIPCION':'descripcion','PRECIO_NETO_LOC':'precio_neto_loc','PRECIO_BRUTO_LOC':'precio_bruto_loc','IMPUESTO_LOC':'impuesto_loc','DESCUENTO_LOC':'descuento_loc'})
marca4['marca'] = 'marca4' 
marca4['anio_entero'] = marca4['fecha_de_venta'].apply(sacar_anio)
marca4['mes_entero'] = marca4['fecha_de_venta'].apply(sacar_mes)
marca4['dia_entero'] = marca4['fecha_de_venta'].apply(sacar_dia)
marca4['mes_string'] = marca4['fecha_de_venta'].dt.strftime('%B')
marca4['semana'] = marca4['fecha_de_venta'].dt.week
marca4['hora_minutos'] = marca4['hora_minutos'].apply(obt_horas_minutos_marca4)

In [13]:
marca4['ventas_sin_iva'] = round(marca4['precio_neto_loc'] * marca4['unidades'],0)
marca4['impuesto_loc'] = round(marca4['impuesto_loc'],0)

In [14]:
marca4['categoria']=marca4['categoria'].str.lower()

In [15]:
marca4['categoria'].unique()

array(['alimentos y bebidas retail', 'golosinas', 'cuidado personal',
       'souvenirs', 'textiles de regalo', 'cafe', 'artesanias', nan,
       'vicios', 'juegos y juguetes', 'joyeria', 'articulos de viaje',
       'libros', 'revistas y periodicos', 'productos de papel',
       'reposteria dulce', 'comida', 'tecnologia', 'complementos cafe',
       'textiles de playa'], dtype=object)

In [16]:
marca4_dummies_cat = pd.get_dummies(marca4.categoria)
marca4_dummies_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037749 entries, 0 to 1037748
Data columns (total 19 columns):
 #   Column                      Non-Null Count    Dtype
---  ------                      --------------    -----
 0   alimentos y bebidas retail  1037749 non-null  uint8
 1   artesanias                  1037749 non-null  uint8
 2   articulos de viaje          1037749 non-null  uint8
 3   cafe                        1037749 non-null  uint8
 4   comida                      1037749 non-null  uint8
 5   complementos cafe           1037749 non-null  uint8
 6   cuidado personal            1037749 non-null  uint8
 7   golosinas                   1037749 non-null  uint8
 8   joyeria                     1037749 non-null  uint8
 9   juegos y juguetes           1037749 non-null  uint8
 10  libros                      1037749 non-null  uint8
 11  productos de papel          1037749 non-null  uint8
 12  reposteria dulce            1037749 non-null  uint8
 13  revistas y periodicos      

In [17]:
marca4_dummies_cat.columns

Index(['alimentos y bebidas retail', 'artesanias', 'articulos de viaje',
       'cafe', 'comida', 'complementos cafe', 'cuidado personal', 'golosinas',
       'joyeria', 'juegos y juguetes', 'libros', 'productos de papel',
       'reposteria dulce', 'revistas y periodicos', 'souvenirs', 'tecnologia',
       'textiles de playa', 'textiles de regalo', 'vicios'],
      dtype='object')

In [18]:
marca4=pd.concat([marca4,marca4_dummies_cat],axis=1)
marca4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037749 entries, 0 to 1037748
Data columns (total 41 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   tienda                      1037749 non-null  object        
 1   muelle                      1037749 non-null  object        
 2   fecha_de_venta              1037749 non-null  datetime64[ns]
 3   hora_minutos                1037749 non-null  object        
 4   hora_entera                 1037749 non-null  int64         
 5   num_fact                    1037749 non-null  int64         
 6   transacciones               1037749 non-null  int64         
 7   descripcion                 1037749 non-null  object        
 8   unidades                    1037749 non-null  int64         
 9   precio_neto_loc             1037749 non-null  float64       
 10  precio_bruto_loc            1037749 non-null  float64       
 11  impuesto_loc            

In [19]:
# marca4_dummies_desc = pd.get_dummies(marca4.descripcion)
# marca4_dummies_desc.info()

In [20]:
marca4_agrupado=marca4.groupby(by=['tienda','anio_entero','mes_entero','dia_entero','hora_entera']).sum()[['ventas_sin_iva','precio_bruto_loc','impuesto_loc','descuento_loc','unidades','ventas','transacciones','alimentos y bebidas retail', 'artesanias', 'articulos de viaje',
       'cafe', 'comida', 'complementos cafe', 'cuidado personal', 'golosinas',
       'joyeria', 'juegos y juguetes', 'libros', 'productos de papel',
       'reposteria dulce', 'revistas y periodicos', 'souvenirs', 'tecnologia',
       'textiles de playa', 'textiles de regalo', 'vicios']].reset_index()

In [79]:
columnas_esp=['tienda','muelle','fecha_de_venta','hora_entera','marca','anio_entero','mes_entero','dia_entero','mes_string','semana']

In [80]:
marca4_conglomerado= marca4.drop_duplicates(subset=['tienda','anio_entero','mes_entero','dia_entero','hora_entera'])[columnas_esp].merge(marca4_agrupado,on=['tienda','anio_entero','mes_entero','dia_entero','hora_entera'],how="left")

In [81]:
marca4_conglomerado=marca4_conglomerado.rename(columns={'precio_bruto_loc':'precio_bruto_total','impuesto_loc':'impuesto_total','descuento_loc':'descuento_total'})

In [82]:
# marca4_conglomerado['categoria']= pd.Series(categoria_col)
# marca4_conglomerado['descripcion'] = pd.Series(descripcion_col)

In [83]:
marca4_conglomerado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17766 entries, 0 to 17765
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tienda                      17766 non-null  object        
 1   muelle                      17766 non-null  object        
 2   fecha_de_venta              17766 non-null  datetime64[ns]
 3   hora_entera                 17766 non-null  int64         
 4   marca                       17766 non-null  object        
 5   anio_entero                 17766 non-null  int64         
 6   mes_entero                  17766 non-null  int64         
 7   dia_entero                  17766 non-null  int64         
 8   mes_string                  17766 non-null  object        
 9   semana                      17766 non-null  int64         
 10  ventas_sin_iva              17766 non-null  float64       
 11  precio_bruto_total          17766 non-null  float64   

In [84]:
marca4_conglomerado.head()

,tienda,muelle,fecha_de_venta,hora_entera,marca,anio_entero,mes_entero,dia_entero,mes_string,semana,...,juegos y juguetes,libros,productos de papel,reposteria dulce,revistas y periodicos,souvenirs,tecnologia,textiles de playa,textiles de regalo,vicios
0,Tienda 1,Internacional Oriente,2022-01-01,0,marca4,2022,1,1,enero,52,...,0,0,0,0,0,5,0,0,2,1
1,Tienda 1,Internacional Oriente,2022-01-01,1,marca4,2022,1,1,enero,52,...,1,0,0,0,0,1,0,0,0,0
2,Tienda 1,Internacional Oriente,2022-01-01,3,marca4,2022,1,1,enero,52,...,1,0,0,0,0,8,0,0,2,0
3,Tienda 1,Internacional Oriente,2022-01-01,4,marca4,2022,1,1,enero,52,...,0,0,0,0,0,5,0,0,0,0
4,Tienda 1,Internacional Oriente,2022-01-01,5,marca4,2022,1,1,enero,52,...,1,0,0,0,0,1,0,0,0,0


In [27]:
marca3=archivo_marca2_10['Marca 3'].copy()
marca3=marca3.drop(columns=['Unnamed: 9','Unnamed: 10'])
marca3=marca3.rename(columns={'FECHA':'fecha_de_venta','Cód tienda':'tienda','VENTAS':'ventas','UBICACIÓN':'muelle','Hora entera':'hora_entera','TRANSACCIONES':'transacciones','UNIDADES':'unidades','CATEGORÍA':'categoria','Tipo de tienda':'tipo_de_tienda'})
marca3= marca3[~(marca3['ventas']==0)]
marca3['marca'] = 'marca3' 
marca3['anio_entero'] = marca3['fecha_de_venta'].apply(sacar_anio)
marca3['mes_entero'] = marca3['fecha_de_venta'].apply(sacar_mes)
marca3['dia_entero'] = marca3['fecha_de_venta'].apply(sacar_dia)
marca3['mes_string'] = marca3['fecha_de_venta'].dt.strftime('%B')
marca3['semana'] = marca3['fecha_de_venta'].dt.week
marca3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235931 entries, 0 to 236964
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   tienda          235931 non-null  object        
 1   tipo_de_tienda  235931 non-null  object        
 2   muelle          235931 non-null  object        
 3   fecha_de_venta  235931 non-null  datetime64[ns]
 4   ventas          235931 non-null  float64       
 5   hora_entera     235931 non-null  int64         
 6   transacciones   235931 non-null  int64         
 7   unidades        235931 non-null  float64       
 8   categoria       235931 non-null  object        
 9   marca           235931 non-null  object        
 10  anio_entero     235931 non-null  int64         
 11  mes_entero      235931 non-null  int64         
 12  dia_entero      235931 non-null  int64         
 13  mes_string      235931 non-null  object        
 14  semana          235931 non-null  int

In [28]:
marca3['categoria']=marca3['categoria'].str.lower()

In [29]:
# 'alimentos y bebidas retail', 'golosinas', 'cuidado personal',
#        'souvenirs', 'textiles de regalo', 'cafe', 'artesanias', nan,
#        'vicios', 'juegos y juguetes', 'joyeria', 'articulos de viaje',
#        'libros', 'revistas y periodicos', 'productos de papel',
#        'reposteria dulce', 'comida', 'tecnologia', 'complementos cafe',
#        'textiles de playa'

In [30]:
def cambiar_categoria_marca3(entrada):
    if entrada=="textiles":
        return "textiles de regalo"
    elif entrada=="tecno":
        return "tecnologia"
    elif entrada=="librería":
        return "libros"
    elif entrada=="belleza":
        return "cuidado personal"
    elif entrada=="alimentos":
        return "comida"
    elif entrada=="licor":
        return "vicios"
    elif entrada=="juguetes":
        return "juegos y juguetes"
    else:
        return entrada

In [31]:
marca3['categoria']= marca3['categoria'].apply(cambiar_categoria_marca3)

In [32]:
marca3['categoria'].unique()

array(['textiles de regalo', 'lujo', 'libros', 'cuidado personal',
       'souvenirs', 'comida', 'accesorios', 'tecnologia', 'perfumería',
       'vicios', 'golosinas', 'juegos y juguetes', 'miscelanea'],
      dtype=object)

In [33]:
marca3_dummies_cat = pd.get_dummies(marca3.categoria)
marca3_dummies_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235931 entries, 0 to 236964
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype
---  ------              --------------   -----
 0   accesorios          235931 non-null  uint8
 1   comida              235931 non-null  uint8
 2   cuidado personal    235931 non-null  uint8
 3   golosinas           235931 non-null  uint8
 4   juegos y juguetes   235931 non-null  uint8
 5   libros              235931 non-null  uint8
 6   lujo                235931 non-null  uint8
 7   miscelanea          235931 non-null  uint8
 8   perfumería          235931 non-null  uint8
 9   souvenirs           235931 non-null  uint8
 10  tecnologia          235931 non-null  uint8
 11  textiles de regalo  235931 non-null  uint8
 12  vicios              235931 non-null  uint8
dtypes: uint8(13)
memory usage: 4.7 MB


In [34]:
marca3=pd.concat([marca3,marca3_dummies_cat],axis=1)
marca3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235931 entries, 0 to 236964
Data columns (total 28 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   tienda              235931 non-null  object        
 1   tipo_de_tienda      235931 non-null  object        
 2   muelle              235931 non-null  object        
 3   fecha_de_venta      235931 non-null  datetime64[ns]
 4   ventas              235931 non-null  float64       
 5   hora_entera         235931 non-null  int64         
 6   transacciones       235931 non-null  int64         
 7   unidades            235931 non-null  float64       
 8   categoria           235931 non-null  object        
 9   marca               235931 non-null  object        
 10  anio_entero         235931 non-null  int64         
 11  mes_entero          235931 non-null  int64         
 12  dia_entero          235931 non-null  int64         
 13  mes_string          235931 no

In [35]:
cols_groupby=['tienda','anio_entero','mes_entero','dia_entero','hora_entera']
cols_merge=['tienda','anio_entero','mes_entero','dia_entero','hora_entera','semana','tipo_de_tienda','fecha_de_venta','marca','mes_string']

In [36]:
marca3_conglomerado=marca3.groupby(by=cols_groupby).sum().drop(columns=['semana']).reset_index().merge(marca3.drop_duplicates(subset=cols_groupby)[cols_merge],on=cols_groupby,how="left").copy()
marca3_conglomerado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79358 entries, 0 to 79357
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tienda              79358 non-null  object        
 1   anio_entero         79358 non-null  int64         
 2   mes_entero          79358 non-null  int64         
 3   dia_entero          79358 non-null  int64         
 4   hora_entera         79358 non-null  int64         
 5   ventas              79358 non-null  float64       
 6   transacciones       79358 non-null  int64         
 7   unidades            79358 non-null  float64       
 8   accesorios          79358 non-null  uint8         
 9   comida              79358 non-null  uint8         
 10  cuidado personal    79358 non-null  uint8         
 11  golosinas           79358 non-null  uint8         
 12  juegos y juguetes   79358 non-null  uint8         
 13  libros              79358 non-null  uint8     

In [37]:
marca3_conglomerado.head()

,tienda,anio_entero,mes_entero,dia_entero,hora_entera,ventas,transacciones,unidades,accesorios,comida,...,perfumería,souvenirs,tecnologia,textiles de regalo,vicios,semana,tipo_de_tienda,fecha_de_venta,marca,mes_string
0,Tienda 1,2022,1,1,6,157818326.0,34,61.0,0,0,...,1,1,1,0,1,52,DUTY FREE,2022-01-01,marca3,enero
1,Tienda 1,2022,1,1,7,120266134.6,26,41.0,0,1,...,1,1,1,0,1,52,DUTY FREE,2022-01-01,marca3,enero
2,Tienda 1,2022,1,1,8,18215139.6,8,22.0,0,0,...,0,1,1,0,1,52,DUTY FREE,2022-01-01,marca3,enero
3,Tienda 1,2022,1,1,9,24501991.4,8,16.0,0,0,...,1,0,1,0,1,52,DUTY FREE,2022-01-01,marca3,enero
4,Tienda 1,2022,1,1,10,123980078.8,23,56.0,0,0,...,1,1,1,1,1,52,DUTY FREE,2022-01-01,marca3,enero


In [38]:
marca5=archivo_marca2_10['Marca 5'].copy()
marca5=marca5.rename(columns={'Fecha':'fecha_de_venta','Ventas':'ventas','Hora entera':'hora_entera','Transacciones':'transacciones','Unidades':'unidades','Muelle':'muelle','DESTINO ':'destino','AEROLINEA':'aerolinea','Descripción del producto':'descripcion'})
marca5['marca'] = 'marca5'
marca5['anio_entero'] = marca5['fecha_de_venta'].apply(sacar_anio)
marca5['mes_entero'] = marca5['fecha_de_venta'].apply(sacar_mes)
marca5['dia_entero'] = marca5['fecha_de_venta'].apply(sacar_dia)
marca5['mes_string'] = marca5['fecha_de_venta'].dt.strftime('%B')
marca5['semana'] = marca5['fecha_de_venta'].dt.week
marca5['tienda'] = 'marca 5' 
marca5['descripcion'] = marca5['descripcion'].str.lower()
marca5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10972 entries, 0 to 10971
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   fecha_de_venta  10972 non-null  datetime64[ns]
 1   hora_entera     10972 non-null  int64         
 2   ventas          10972 non-null  int64         
 3   transacciones   10972 non-null  int64         
 4   unidades        10972 non-null  int64         
 5   descripcion     10961 non-null  object        
 6   destino         10918 non-null  object        
 7   aerolinea       10909 non-null  object        
 8   muelle          10972 non-null  object        
 9   marca           10972 non-null  object        
 10  anio_entero     10972 non-null  int64         
 11  mes_entero      10972 non-null  int64         
 12  dia_entero      10972 non-null  int64         
 13  mes_string      10972 non-null  object        
 14  semana          10972 non-null  int64         
 15  ti

In [223]:
categorias=['almohada','billetera','bolso','bolso de viaje','cafe','chaqueta','cosmetiquero','zapato','media','accesorio','morral','mocasin','gorra beisbolera','gafa','sandalia','gorra','tarjetero','ruana','pañoleta','camiseta','tacon','aguadeño','aerpos',
           'tapaboca','libro','baul','cinturon','correa','capa','cambio','canguro','carriel','casa hogar','cardigan','cartuchera','estuche','chal','maleta','maleta de viaje','suspiro','fragancia','manos libres','llavero','descripcion','alimento','oficina',
           'brazelete','monedero','neceser','obsequio','sombrero','prenda','papelera','esmeralda','joya','tarjetero','tula herencia','tejido','riñonera','shoulder','botella','hebilla','tira','faja de seguridad','calzado','cruzado','portadocumentos','porta pasaporte','porta pc',
           'marroquineria','limpiador','linea ejecutiva','linea de viaje','pulsera','lederter band','escritorio']

In [224]:
# se pueden primero normalizar los nombres de cada cosa aunque este separada , pero además si es más de un objeto se pueden separar por columnas y luego sumar esas columnas dummies generadas que sean iguales y ahí si quedar con unas
# columnas dummies bien generadas y limpías sin tantas categorías.

In [225]:
marca5['descripcion']=marca5['descripcion'].str.replace("-",'+')
marca5['descripcion']=marca5['descripcion'].str.replace(",",'+')
marca5['descripcion']=marca5['descripcion'].str.replace("y",'+')
marca5['descripcion']=marca5['descripcion'].str.strip()

In [226]:
list_objetos=[[],[],[],[],[]]
for descripcion in marca5['descripcion']:
    if isinstance(descripcion,float):
        list_objetos[0].append("")
        list_objetos[1].append("")
        list_objetos[2].append("")
        list_objetos[3].append("")
        list_objetos[4].append("")
        continue
    descripcion_list=descripcion.split("+")
    cont=0
    for descripcion_unica in descripcion_list:
        lista_valores=[]
        for categoria in categorias:
            lista_valores.append(SequenceMatcher(None, descripcion_unica, categoria).ratio())
        valor_max=max(lista_valores)
        posicion_lista=lista_valores.index(valor_max)
        list_objetos[cont].append(categorias[posicion_lista])
        cont= cont+1
    if len(descripcion_list)==0:
        list_objetos[0].append("")
        list_objetos[1].append("")
        list_objetos[2].append("")
        list_objetos[3].append("")
        list_objetos[4].append("")
    elif len(descripcion_list)==1:
        list_objetos[1].append("")
        list_objetos[2].append("")
        list_objetos[3].append("")
        list_objetos[4].append("")
    elif len(descripcion_list)==2:
        list_objetos[2].append("")
        list_objetos[3].append("")
        list_objetos[4].append("")
    elif len(descripcion_list)==3:
        list_objetos[3].append("")
        list_objetos[4].append("")
    elif len(descripcion_list)==4:
        list_objetos[4].append("")
    
    

In [227]:
len(list_objetos[0])

10972

In [228]:
marca5['objeto1']=list_objetos[0]
marca5['objeto2']=list_objetos[1]
marca5['objeto3']=list_objetos[2]
marca5['objeto4']=list_objetos[3]
marca5['objeto5']=list_objetos[4]

In [ ]:
# Falta generar dummies de cada fila de objeto , sumarlas, y dejar las columnas restantes como las dummies de la marca 5.

In [229]:
# marca5.sort_values(by="descripcion")['descripcion'].unique()

In [230]:
# marca5.drop_duplicates(subset=['descripcion'])[['descripcion','objeto1','objeto2','objeto3','objeto4','objeto5']].to_excel("ejemplo_cat_marca5.xlsx")

In [ ]:
# Falta generar

In [196]:
marca5_fin=marca5.drop(columns=['destino','aerolinea']).copy()
marca5_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10972 entries, 0 to 10971
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   fecha_de_venta  10972 non-null  datetime64[ns]
 1   hora_entera     10972 non-null  int64         
 2   ventas          10972 non-null  int64         
 3   transacciones   10972 non-null  int64         
 4   unidades        10972 non-null  int64         
 5   descripcion     10961 non-null  object        
 6   muelle          10972 non-null  object        
 7   marca           10972 non-null  object        
 8   anio_entero     10972 non-null  int64         
 9   mes_entero      10972 non-null  int64         
 10  dia_entero      10972 non-null  int64         
 11  mes_string      10972 non-null  object        
 12  semana          10972 non-null  int64         
 13  tienda          10972 non-null  object        
 14  objeto1         10972 non-null  object        
 15  ob

In [44]:
marca6=archivo_marca2_10['Marca 6'].copy()
marca6=marca6.rename(columns={'Fecha':'fecha_de_venta','Ventas':'ventas','Hora de venta':'hora_entera','Transacciones':'transacciones','Unidades':'unidades','Muelle':'muelle','Categoría':'categoria','Subcategoría':'subcategoria'})
marca6= marca6[~(marca6['ventas']==0)]
marca6['marca'] = 'marca6'
marca6['anio_entero'] = marca6['fecha_de_venta'].apply(sacar_anio)
marca6['mes_entero'] = marca6['fecha_de_venta'].apply(sacar_mes)
marca6['dia_entero'] = marca6['fecha_de_venta'].apply(sacar_dia)
marca6['mes_string'] = marca6['fecha_de_venta'].dt.strftime('%B')
marca6['semana'] = marca6['fecha_de_venta'].dt.week
marca6['tienda'] = 'marca 6' 
marca6['categoria'] = marca6['categoria'] + " " + marca6['subcategoria']
marca6=marca6.drop(columns=['subcategoria'])
marca6['categoria']= marca6['categoria'].str.lower()
marca6['transacciones']= marca6['transacciones'].astype(int)
marca6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34669 entries, 0 to 43794
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   fecha_de_venta  34669 non-null  datetime64[ns]
 1   categoria       34669 non-null  object        
 2   hora_entera     34669 non-null  float64       
 3   ventas          34669 non-null  int64         
 4   unidades        34669 non-null  float64       
 5   transacciones   34669 non-null  int32         
 6   muelle          34669 non-null  object        
 7   marca           34669 non-null  object        
 8   anio_entero     34669 non-null  int64         
 9   mes_entero      34669 non-null  int64         
 10  dia_entero      34669 non-null  int64         
 11  mes_string      34669 non-null  object        
 12  semana          34669 non-null  int64         
 13  tienda          34669 non-null  object        
dtypes: datetime64[ns](1), float64(2), int32(1), int64(5), 

In [45]:
marca6['categoria'].unique()

array(['calzado masculino', 'calzado femenino', 'cinturones masculino',
       'cinturones femenino', 'marroquinería masculino',
       'marroquinería femenino', 'bolsos masculino', 'bolsos femenino',
       'prendas de vestir masculino', 'prendas de vestir femenino',
       'productos comerciali unisex', 'accesorios de lujo masculino',
       'travel & business masculino', 'travel & business femenino',
       'accesorios de lujo femenino', 'bienestar masculino',
       'bienestar femenino', 'sin asignar sin asignar',
       'bienestar unisex'], dtype=object)

In [46]:
marca6['transacciones'].unique()

array([0])

In [47]:
marca6_dummies_cat = pd.get_dummies(marca6.categoria)
marca6_dummies_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34669 entries, 0 to 43794
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   accesorios de lujo femenino   34669 non-null  uint8
 1   accesorios de lujo masculino  34669 non-null  uint8
 2   bienestar femenino            34669 non-null  uint8
 3   bienestar masculino           34669 non-null  uint8
 4   bienestar unisex              34669 non-null  uint8
 5   bolsos femenino               34669 non-null  uint8
 6   bolsos masculino              34669 non-null  uint8
 7   calzado femenino              34669 non-null  uint8
 8   calzado masculino             34669 non-null  uint8
 9   cinturones femenino           34669 non-null  uint8
 10  cinturones masculino          34669 non-null  uint8
 11  marroquinería femenino        34669 non-null  uint8
 12  marroquinería masculino       34669 non-null  uint8
 13  prendas de vestir femenino    3

In [48]:
marca6=pd.concat([marca6,marca6_dummies_cat],axis=1)
marca6.iloc[:,:20].head()

,fecha_de_venta,categoria,hora_entera,ventas,unidades,transacciones,muelle,marca,anio_entero,mes_entero,dia_entero,mes_string,semana,tienda,accesorios de lujo femenino,accesorios de lujo masculino,bienestar femenino,bienestar masculino,bienestar unisex,bolsos femenino
0,2022-01-02,calzado masculino,7.0,3478800,1.0,0,A Internacional Oriente,marca6,2022,1,2,enero,52,marca 6,0,0,0,0,0,0
1,2022-01-02,calzado masculino,8.0,4438800,1.0,0,A Internacional Oriente,marca6,2022,1,2,enero,52,marca 6,0,0,0,0,0,0
2,2022-01-02,calzado masculino,14.0,10076400,3.0,0,A Internacional Oriente,marca6,2022,1,2,enero,52,marca 6,0,0,0,0,0,0
3,2022-01-02,calzado masculino,16.0,19113600,4.0,0,A Internacional Oriente,marca6,2022,1,2,enero,52,marca 6,0,0,0,0,0,0
4,2022-01-02,calzado masculino,17.0,3838800,1.0,0,A Internacional Oriente,marca6,2022,1,2,enero,52,marca 6,0,0,0,0,0,0


In [49]:
marca6['transacciones'].unique()

array([0])

In [50]:
# marca10.iloc[:,:2]

In [51]:
cols_groupby_6=['anio_entero','mes_entero','dia_entero','hora_entera']
cols_merge_6=['tienda','anio_entero','mes_entero','dia_entero','hora_entera','semana','fecha_de_venta','marca','mes_string']

In [52]:
marca6_conglomerado=marca6.groupby(by=cols_groupby_6).sum().drop(columns=['semana']).reset_index().merge(marca6.drop_duplicates(subset=cols_groupby_6)[cols_merge_6],how="left",on=cols_groupby_6)
marca6_conglomerado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9116 entries, 0 to 9115
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   anio_entero                   9116 non-null   int64         
 1   mes_entero                    9116 non-null   int64         
 2   dia_entero                    9116 non-null   int64         
 3   hora_entera                   9116 non-null   float64       
 4   ventas                        9116 non-null   int64         
 5   unidades                      9116 non-null   float64       
 6   transacciones                 9116 non-null   int32         
 7   accesorios de lujo femenino   9116 non-null   uint8         
 8   accesorios de lujo masculino  9116 non-null   uint8         
 9   bienestar femenino            9116 non-null   uint8         
 10  bienestar masculino           9116 non-null   uint8         
 11  bienestar unisex              

In [53]:
marca6_conglomerado.sample(10)

,anio_entero,mes_entero,dia_entero,hora_entera,ventas,unidades,transacciones,accesorios de lujo femenino,accesorios de lujo masculino,bienestar femenino,...,prendas de vestir masculino,productos comerciali unisex,sin asignar sin asignar,travel & business femenino,travel & business masculino,tienda,semana,fecha_de_venta,marca,mes_string
6965,2023,3,22,14.0,52584000,8.0,0,0,0,0,...,1,0,0,0,0,marca 6,12,2023-03-22,marca6,marzo
1504,2022,4,11,6.0,52480000,10.0,0,0,0,0,...,1,1,0,0,0,marca 6,15,2022-04-11,marca6,abril
6334,2023,2,12,22.0,33988000,6.0,0,0,0,0,...,0,0,0,0,0,marca 6,6,2023-02-12,marca6,febrero
981,2022,3,7,6.0,13192000,4.0,0,1,0,0,...,0,0,0,0,0,marca 6,10,2022-03-07,marca6,marzo
7134,2023,4,2,23.0,4998000,1.0,0,0,0,0,...,1,0,0,0,0,marca 6,13,2023-04-02,marca6,abril
4435,2022,10,13,7.0,28992000,4.0,0,0,0,0,...,0,0,0,0,0,marca 6,41,2022-10-13,marca6,octubre
2978,2022,7,13,21.0,6396000,2.0,0,0,0,0,...,1,0,0,0,0,marca 6,28,2022-07-13,marca6,julio
4123,2022,9,23,13.0,15394000,3.0,0,0,0,0,...,1,0,0,0,0,marca 6,38,2022-09-23,marca6,septiembre
3774,2022,8,31,22.0,5796000,2.0,0,0,0,0,...,0,0,0,0,0,marca 6,35,2022-08-31,marca6,agosto
136,2022,1,10,16.0,17392000,4.0,0,0,0,0,...,0,0,0,0,0,marca 6,2,2022-01-10,marca6,enero


In [54]:
marca6_conglomerado['transacciones'].unique()

array([0])

In [55]:
marca7=archivo_marca2_10['Marca 7'].copy()
marca7=marca7.rename(columns={'Fecha':'fecha_de_venta','Venta':'ventas','Hora':'hora_entera','Transacciones':'transacciones','Unidades':'unidades','Muelle':'muelle','Personas que ingresan':'personas_que_ingresan'})
marca7= marca7[~(marca7['ventas']==0.0)]
marca7['marca'] = 'marca7'
marca7['anio_entero'] = marca7['fecha_de_venta'].apply(sacar_anio)
marca7['mes_entero'] = marca7['fecha_de_venta'].apply(sacar_mes)
marca7['dia_entero'] = marca7['fecha_de_venta'].apply(sacar_dia)
marca7['mes_string'] = marca7['fecha_de_venta'].dt.strftime('%B')
marca7['semana'] = marca7['fecha_de_venta'].dt.week
marca7['tienda'] = 'marca 7' 
marca7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7947 entries, 0 to 11611
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   fecha_de_venta         7947 non-null   datetime64[ns]
 1   hora_entera            7947 non-null   float64       
 2   personas_que_ingresan  7947 non-null   float64       
 3   ventas                 7947 non-null   float64       
 4   transacciones          7947 non-null   float64       
 5   unidades               7947 non-null   float64       
 6   muelle                 7947 non-null   object        
 7   marca                  7947 non-null   object        
 8   anio_entero            7947 non-null   int64         
 9   mes_entero             7947 non-null   int64         
 10  dia_entero             7947 non-null   int64         
 11  mes_string             7947 non-null   object        
 12  semana                 7947 non-null   int64         
 13  ti

In [56]:
marca7.head()

,fecha_de_venta,hora_entera,personas_que_ingresan,ventas,transacciones,unidades,muelle,marca,anio_entero,mes_entero,dia_entero,mes_string,semana,tienda
0,2022-01-01,7.0,13.0,7393277.6,4.0,12.0,A Internacional oriente,marca7,2022,1,1,enero,52,marca 7
1,2022-01-01,8.0,5.0,3574790.0,1.0,4.0,A Internacional oriente,marca7,2022,1,1,enero,52,marca 7
3,2022-01-01,10.0,2.0,1610084.0,1.0,2.0,A Internacional oriente,marca7,2022,1,1,enero,52,marca 7
4,2022-01-01,11.0,16.0,4968067.2,2.0,6.0,A Internacional oriente,marca7,2022,1,1,enero,52,marca 7
5,2022-01-01,12.0,9.0,1628571.4,1.0,1.0,A Internacional oriente,marca7,2022,1,1,enero,52,marca 7


In [57]:
marca8=archivo_marca2_10['marca 8'].copy()
marca8=marca8.drop(columns=['Area','Unnamed: 10','Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19','Unnamed: 20','Unnamed: 21','Unnamed: 22','Unnamed: 23','Unnamed: 24','Unnamed: 25'])
marca8=marca8.rename(columns={'Fecha':'fecha_de_venta','Tienda':'tienda','Ventas':'ventas','Hora entera':'hora_entera','Transacciones':'transacciones','Unidades':'unidades','# Semana':'semana','Año':'anio_entero','Mes':'mes_string'})
marca8['marca'] = 'marca8'
marca8['anio_entero'] = marca8['fecha_de_venta'].apply(sacar_anio)
marca8['mes_entero'] = marca8['fecha_de_venta'].apply(sacar_mes)
marca8['dia_entero'] = marca8['fecha_de_venta'].apply(sacar_dia)
marca8['mes_string'] = marca8['fecha_de_venta'].dt.strftime('%B')
marca8['semana'] = marca8['fecha_de_venta'].dt.week
marca8['hora_entera']= marca8['hora_entera'].astype(str)
marca8['hora_minutos'] = marca8['hora_entera']
marca8['hora_entera'] = marca8['hora_entera'].apply(obt_horas_marca8)
marca8['hora_minutos'] = marca8['hora_minutos'].apply(obt_horas_minutos_marca8)
marca8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tienda          2759 non-null   object        
 1   anio_entero     2762 non-null   float64       
 2   mes_string      2762 non-null   object        
 3   fecha_de_venta  2762 non-null   datetime64[ns]
 4   hora_entera     2930 non-null   object        
 5   semana          2762 non-null   float64       
 6   ventas          2761 non-null   float64       
 7   transacciones   2762 non-null   float64       
 8   unidades        2761 non-null   object        
 9   marca           2930 non-null   object        
 10  mes_entero      2762 non-null   float64       
 11  dia_entero      2762 non-null   float64       
 12  hora_minutos    2930 non-null   object        
dtypes: datetime64[ns](1), float64(6), object(6)
memory usage: 297.7+ KB


In [58]:
cols_groupby_8=['tienda','anio_entero','mes_entero','dia_entero','hora_entera']
cols_merge_8=['tienda','anio_entero','mes_entero','dia_entero','hora_entera','semana','fecha_de_venta','marca','mes_string']

In [59]:
marca8_conglomerado=marca8.groupby(by=cols_groupby_8).sum().drop(columns=['semana']).reset_index().merge(marca8.drop_duplicates(subset=cols_groupby_8)[cols_merge_8],how="left",on=cols_groupby_8)
marca8_conglomerado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2188 entries, 0 to 2187
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tienda          2188 non-null   object        
 1   anio_entero     2188 non-null   float64       
 2   mes_entero      2188 non-null   float64       
 3   dia_entero      2188 non-null   float64       
 4   hora_entera     2188 non-null   object        
 5   ventas          2188 non-null   float64       
 6   transacciones   2188 non-null   float64       
 7   semana          2188 non-null   float64       
 8   fecha_de_venta  2188 non-null   datetime64[ns]
 9   marca           2188 non-null   object        
 10  mes_string      2188 non-null   object        
dtypes: datetime64[ns](1), float64(6), object(4)
memory usage: 205.1+ KB


In [60]:
marca8_conglomerado.head()

,tienda,anio_entero,mes_entero,dia_entero,hora_entera,ventas,transacciones,semana,fecha_de_venta,marca,mes_string
0,MARCA 8,2002.0,9.0,27.0,19,22000000.0,1.0,39.0,2002-09-27,marca8,septiembre
1,MARCA 8,2020.0,1.0,9.0,13,64000000.0,1.0,2.0,2020-01-09,marca8,enero
2,MARCA 8,2020.0,3.0,29.0,12,17000000.0,1.0,13.0,2020-03-29,marca8,marzo
3,MARCA 8,2021.0,1.0,3.0,14,64400000.0,2.0,53.0,2021-01-03,marca8,enero
4,MARCA 8,2021.0,1.0,3.0,18,55200000.0,1.0,53.0,2021-01-03,marca8,enero


In [61]:
marca9= archivo_marca2_10['marca 9'].copy()
marca9=marca9.drop(columns=['Unnamed: 10','Unnamed: 11','Unnamed: 12'])
marca9=marca9.rename(columns={'Fecha':'fecha_de_venta','TIenda':'tienda','Ventas sin IVA ':'ventas_sin_iva','Hora entera':'hora_entera','Transacciones':'transacciones','Unidades':'unidades','# Semana':'semana','Año':'anio_entero','Mes':'mes_string','Muelle':'muelle','Piezas':'unidades','Tickets':'transacciones'})
marca9= marca9[~(marca9['ventas_sin_iva']==0)]
marca9['marca'] = 'marca9'
marca9['anio_entero'] = marca9['fecha_de_venta'].apply(sacar_anio)
marca9['mes_entero'] = marca9['fecha_de_venta'].apply(sacar_mes)
marca9['dia_entero'] = marca9['fecha_de_venta'].apply(sacar_dia)
marca9['mes_string'] = marca9['fecha_de_venta'].dt.strftime('%B')
marca9['semana'] = marca9['fecha_de_venta'].dt.week
marca9.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11174 entries, 0 to 11254
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tienda          11174 non-null  object        
 1   anio_entero     11174 non-null  int64         
 2   mes_string      11174 non-null  object        
 3   fecha_de_venta  11174 non-null  datetime64[ns]
 4   semana          11174 non-null  int64         
 5   hora_entera     11174 non-null  int64         
 6   ventas_sin_iva  11174 non-null  int64         
 7   transacciones   11173 non-null  float64       
 8   unidades        11173 non-null  float64       
 9   muelle          11174 non-null  object        
 10  marca           11174 non-null  object        
 11  mes_entero      11174 non-null  int64         
 12  dia_entero      11174 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(6), object(4)
memory usage: 1.2+ MB


In [62]:
marca9.head()

,tienda,anio_entero,mes_string,fecha_de_venta,semana,hora_entera,ventas_sin_iva,transacciones,unidades,muelle,marca,mes_entero,dia_entero
0,MARCA 9,2022,enero,2022-01-02,52,5,11425940,18.0,40.0,Internacional Oriente,marca9,1,2
1,MARCA 9,2022,enero,2022-01-02,52,6,3070280,6.0,8.0,Internacional Oriente,marca9,1,2
2,MARCA 9,2022,enero,2022-01-02,52,7,13548300,20.0,58.0,Internacional Oriente,marca9,1,2
3,MARCA 9,2022,enero,2022-01-02,52,8,2982360,7.0,11.0,Internacional Oriente,marca9,1,2
4,MARCA 9,2022,enero,2022-01-02,52,9,4048960,6.0,15.0,Internacional Oriente,marca9,1,2


In [63]:
marca10= archivo_marca2_10['marca 10'].copy()
marca10= marca10.drop(columns=['Factura'])
marca10=marca10.rename(columns={'Fecha':'fecha_de_venta','Tienda':'tienda','Ventas':'ventas','Hora entera':'hora_entera','Transacciones':'transacciones','Unidades':'unidades','Muelle':'muelle','Department':'departamento','Category':'categoria','Total (Tax Ex)':'ventas_sin_iva','Hora':'hora_minutos'})
marca10= marca10.drop(columns=['departamento'])
marca10['marca'] = 'marca10'
marca10['anio_entero'] = marca10['fecha_de_venta'].apply(sacar_anio)
marca10['mes_entero'] = marca10['fecha_de_venta'].apply(sacar_mes)
marca10['dia_entero'] = marca10['fecha_de_venta'].apply(sacar_dia)
marca10['mes_string'] = marca10['fecha_de_venta'].dt.strftime('%B')
marca10['semana'] = marca10['fecha_de_venta'].dt.week
marca10['hora_minutos'] = marca10['hora_minutos'].astype(str)
marca10['hora_minutos'] = marca10['hora_minutos'].apply(obt_horas_minutos_marca10)
marca10['ventas_sin_iva']=marca10['ventas_sin_iva'].astype(int)
marca10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4575 entries, 0 to 4574
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   categoria       4575 non-null   object        
 1   unidades        4575 non-null   int64         
 2   ventas          4575 non-null   int64         
 3   ventas_sin_iva  4575 non-null   int32         
 4   transacciones   4575 non-null   int64         
 5   fecha_de_venta  4575 non-null   datetime64[ns]
 6   hora_minutos    4575 non-null   object        
 7   hora_entera     4575 non-null   int64         
 8   muelle          4575 non-null   object        
 9   tienda          4575 non-null   object        
 10  marca           4575 non-null   object        
 11  anio_entero     4575 non-null   int64         
 12  mes_entero      4575 non-null   int64         
 13  dia_entero      4575 non-null   int64         
 14  mes_string      4575 non-null   object        
 15  sema

In [64]:
marca10['categoria']=marca10['categoria'].str.lower()

In [65]:
marca10['categoria'].unique()

array(['categoria 9', 'categoria 1', 'categoria 2', 'categoria 3',
       'categoria 4', 'categoria 5', 'categoria 6', 'categoria 7',
       'categoria 8', 'categoria 10', 'categoria 11', 'categoria 12',
       'categoria 13', 'categoria 14', 'categoria 15', 'categoria 16',
       'categoria 17', 'categoria 18', 'categoria 19', 'categoria 20',
       'categoria 21', 'categoria 22', 'categoria 23', 'categoria 24',
       'categoria 25', 'categoria 26', 'categoria 27', 'categoria 28',
       'categoria 29', 'categoria 30', 'categoria 31', 'categoria 32',
       'categoria 33', 'categoria 34', 'categoria 35', 'categoria 36',
       'categoria 37', 'categoria 38', 'categoria 39', 'categoria 40',
       'categoria 41', 'categoria 42', 'categoria 43', 'categoria 44',
       'categoria 45', 'categoria 46', 'categoria 47', 'categoria 48',
       'categoria 49', 'categoria 50', 'categoria 51', 'categoria 52',
       'categoria 53', 'categoria 54', 'categoria 55', 'categoria 56',
       'categor

In [66]:
marca10_dummies_cat = pd.get_dummies(marca10.categoria)
marca10_dummies_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4575 entries, 0 to 4574
Columns: 103 entries, categoria 1 to categoria 99
dtypes: uint8(103)
memory usage: 460.3 KB


In [67]:
marca10=pd.concat([marca10,marca10_dummies_cat],axis=1)
marca10.iloc[:,:20].head()

,categoria,unidades,ventas,ventas_sin_iva,transacciones,fecha_de_venta,hora_minutos,hora_entera,muelle,tienda,marca,anio_entero,mes_entero,dia_entero,mes_string,semana,categoria 1,categoria 10,categoria 100,categoria 101
0,categoria 9,1,4238400,3561680,1,2022-09-16,54:45,13,A Internacional Oriente,marca 10,marca10,2022,9,16,septiembre,37,0,0,0,0
1,categoria 1,1,2988700,2511512,1,2022-09-16,22:54,14,A Internacional Oriente,marca 10,marca10,2022,9,16,septiembre,37,1,0,0,0
2,categoria 2,1,2398400,2015462,1,2022-09-16,19:46,15,A Internacional Oriente,marca 10,marca10,2022,9,16,septiembre,37,0,0,0,0
3,categoria 3,1,2800000,2352941,0,2022-09-16,19:46,15,A Internacional Oriente,marca 10,marca10,2022,9,16,septiembre,37,0,0,0,0
4,categoria 4,1,2398400,2015462,1,2022-09-16,55:05,19,A Internacional Oriente,marca 10,marca10,2022,9,16,septiembre,37,0,0,0,0


In [68]:
# marca10.iloc[:,:2]

In [69]:
cols_groupby_10=['tienda','anio_entero','mes_entero','dia_entero','hora_entera']
cols_merge_10=['tienda','anio_entero','mes_entero','dia_entero','hora_entera','semana','fecha_de_venta','marca','mes_string']

In [70]:
marca10_conglomerado=marca10.groupby(by=cols_groupby_10).sum().drop(columns=['semana']).reset_index().merge(marca10.drop_duplicates(subset=cols_groupby_10)[cols_merge_10],how="left",on=cols_groupby_10)
marca10_conglomerado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2033 entries, 0 to 2032
Columns: 116 entries, tienda to mes_string
dtypes: datetime64[ns](1), int32(1), int64(8), object(3), uint8(103)
memory usage: 418.9+ KB


In [71]:
marca10_conglomerado.iloc[:,:20].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2033 entries, 0 to 2032
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tienda          2033 non-null   object
 1   anio_entero     2033 non-null   int64 
 2   mes_entero      2033 non-null   int64 
 3   dia_entero      2033 non-null   int64 
 4   hora_entera     2033 non-null   int64 
 5   unidades        2033 non-null   int64 
 6   ventas          2033 non-null   int64 
 7   ventas_sin_iva  2033 non-null   int32 
 8   transacciones   2033 non-null   int64 
 9   categoria 1     2033 non-null   uint8 
 10  categoria 10    2033 non-null   uint8 
 11  categoria 100   2033 non-null   uint8 
 12  categoria 101   2033 non-null   uint8 
 13  categoria 102   2033 non-null   uint8 
 14  categoria 103   2033 non-null   uint8 
 15  categoria 11    2033 non-null   uint8 
 16  categoria 12    2033 non-null   uint8 
 17  categoria 13    2033 non-null   uint8 
 18  categori

In [72]:
marca10_conglomerado.iloc[:,:10].head()

,tienda,anio_entero,mes_entero,dia_entero,hora_entera,unidades,ventas,ventas_sin_iva,transacciones,categoria 1
0,marca 10,2022,9,16,13,1,4238400,3561680,1,0
1,marca 10,2022,9,16,14,1,2988700,2511512,1,1
2,marca 10,2022,9,16,15,2,5198400,4368403,1,0
3,marca 10,2022,9,16,19,1,2398400,2015462,1,0
4,marca 10,2022,9,16,20,4,15278400,12838991,3,0


In [73]:
marca10_conglomerado['transacciones'].unique()

array([1, 3, 5, 2, 4, 6, 7], dtype=int64)

In [85]:
dataframe_unificado= pd.concat([marca1,marca2,marca3_conglomerado,marca4_conglomerado,marca5_fin,marca6_conglomerado,marca7,marca8_conglomerado,marca9,marca10_conglomerado])
dataframe_unificado.iloc[:,:100].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246221 entries, 0 to 2032
Data columns (total 100 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   fecha_de_venta                246221 non-null  datetime64[ns]
 1   tienda                        246221 non-null  object        
 2   ventas                        235047 non-null  float64       
 3   muelle                        153526 non-null  object        
 4   hora_entera                   246133 non-null  object        
 5   transacciones                 246132 non-null  float64       
 6   semana                        246221 non-null  float64       
 7   anio_entero                   246221 non-null  float64       
 8   mes_entero                    246221 non-null  float64       
 9   dia_entero                    246221 non-null  float64       
 10  mes_string                    246221 non-null  object        
 11  marca         

In [86]:
dataframe_unificado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246221 entries, 0 to 2032
Columns: 165 entries, fecha_de_venta to categoria 99
dtypes: datetime64[ns](1), float64(157), object(7)
memory usage: 311.8+ MB


In [95]:
columnas_a_cambiar=list(dataframe_unificado.iloc[:,13:].columns)
columnas_a_cambiar.remove('descripcion')
columnas_a_cambiar.remove('ventas_sin_iva')
columnas_a_cambiar.remove('precio_bruto_total')
columnas_a_cambiar.remove('impuesto_total')
columnas_a_cambiar.remove('descuento_total')
columnas_a_cambiar.remove('personas_que_ingresan')

In [97]:
for col in columnas_a_cambiar:
    dataframe_unificado[col]=  dataframe_unificado[col].fillna(0)

In [98]:
dataframe_unificado.iloc[:,:100].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246221 entries, 0 to 2032
Data columns (total 100 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   fecha_de_venta                246221 non-null  datetime64[ns]
 1   tienda                        246221 non-null  object        
 2   ventas                        235047 non-null  float64       
 3   muelle                        153526 non-null  object        
 4   hora_entera                   246133 non-null  object        
 5   transacciones                 246132 non-null  float64       
 6   semana                        246221 non-null  float64       
 7   anio_entero                   246221 non-null  float64       
 8   mes_entero                    246221 non-null  float64       
 9   dia_entero                    246221 non-null  float64       
 10  mes_string                    246221 non-null  object        
 11  marca         

In [99]:
# dataframe_unificado.to_excel("Marcas1-10Unificado.xlsx")

In [77]:
dataframe_unificado['tienda'].unique()

array(['TIENDA 1', 'TIENDA 4', 'TIENDA 5', 'TIENDA 6', 'TIENDA 3',
       'TIENDA 2', 'TIENDA 12', 'TIENDA 11', 'TIENDA 10 ', 'TIENDA 9',
       'Tienda A', 'Tienda B', 'Tienda 1', 'Tienda 2', 'Tienda 3',
       'Tienda 4', 'Tienda 5', 'Tienda 6', 'Tienda 7', 'Tienda 8',
       'Tienda 9', 'marca 5', 'marca 6', 'marca 7', 'MARCA 8', 'MARCA 9',
       'marca 10'], dtype=object)